#Linear regression

Here we 
* make up a regression model, 
* sample some points from it, and 
* see how well we can recover the true parameters.  

Though it's very simple, this example illustrates basic Gamble usage, including observing functions of random variables, and some iGamble visualization tools.

In [17]:
(require gamble 
         racket/class
         racket/list
         racket/vector
         racket/port
         "lr_helpers.rkt"
         "c3_helpers.rkt")

##Make up a true regression model

We pick a slope, a y-intercept, and an error sigma, which we will use to generate some data.  

$$y_i \sim N(mx_i + b,\sigma)$$

We want to recover the true parameters from the data.

In [6]:
(define true-slope 3)
(define true-intercept 12)
(define true-noise-sigma 15)

##Make some data

We will collect random $y$ values for 30 evenly-spaced $x$.

In [7]:
(define n-points 30)
(define xs (build-vector n-points (lambda (x) x)))
(define ys (build-vector n-points
                         (lambda (x) (+ (* x true-slope) 
                                        true-intercept 
                                        (normal 0 true-noise-sigma)))))

In [8]:
(scatter-c3 (vector->list xs) (vector->list ys) #:legend "sample points")

(c3-data . #hasheq((data . #hasheq((type . scatter) (xs . #hasheq((ys1 . xs1))) (columns . ((xs1 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29) (ys1 36.69636581604345 15.275191354648758 24.603086668962312 29.38871759975995 -6.443376421595538 19.530137501756798 33.29186825447499 38.273456645595545 46.59489863248796 74.11550624081565 68.33421490916538 66.02961631177506 25.130781970508885 65.47977980932727 42.154063939175884 33.2723700424353 40.32061046835154 56.69304459563036 50.370525887923705 43.52332517342687 85.88604840202373 83.82844724158305 73.50519896426346 86.35263412962858 78.19083996643813 64.17075733228134 80.07216982950666 104.52710308236034 112.0610372759869 127.35182140105486))))) (bindto . chart1) (point . #hasheq((show . #t))) (axis . #hasheq((x . #hasheq((label . x) (tick . #hasheq((values . (0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29)))))) (y . #hasheq((label . y)))))))

##Make a generative model inside a sampler

This is the core probabilistic program, here.  Not much to it.

In [9]:
(define lr
  (mh-sampler
   (define slope (normal 0 10))
   (define intercept (normal 0 10))
   (define noise-sigma (gamma 1 15))
   (define (f x)
     (+ (* slope x) intercept (normal 0 noise-sigma)))
   (for ([x xs]
         [y ys])
     (observe (f x) y))
   (vector slope intercept noise-sigma)))

In [10]:
(define results (sample-and-monitor lr 50 #:burn 500 #:thin 50))

In [11]:
(show-acc-rates results)

(c3-data . #hasheq((data . #hasheq((type . line) (xs . #hasheq((proposal acceptance rate . xs1))) (columns . ((xs1 49 99 149 199 249 299 349 399 449 500 551 602 653 704 755 806 857 908 959 1010 1061 1112 1163 1214 1265 1316 1367 1418 1469 1520 1571 1622 1673 1724 1775 1826 1877 1928 1979 2030 2081 2132 2183 2234 2285 2336 2387 2438 2489 2540 2591 2642 2693 2744 2795 2846 2897 2948 2999) (proposal acceptance rate 0.16326530612241566 0.139999999999972 0.159999999999968 0.139999999999972 0.139999999999972 0.19999999999996002 0.119999999999976 0.159999999999968 0.139999999999972 0.1372549019607574 0.21568627450976163 0.21568627450976163 0.17647058823525952 0.25490196078426375 0.1372549019607574 0.17647058823525952 0.05882352941175317 0.11764705882350635 0.21568627450976163 0.3137254901960169 0.3725490196077701 0.2745098039215148 0.09803921568625529 0.2352941176470127 0.17647058823525952 0.25490196078426375 0.2352941176470127 0.19607843137251058 0.25490196078426375 0.09803921568625529 0.274

In [12]:
(show-scale-changes results)

(c3-data . #hasheq((data . #hasheq((type . line) (xs . #hasheq((increase . xs0) (decrease . xs1) (nochange . xs2))) (columns . ((xs0 49 99 149 199 249 299 349 399 449 500 551 602 653 704 755 806 857 908 959 1010 1061 1112 1163 1214 1265 1316 1367 1418 1469 1520 1571 1622 1673 1724 1775 1826 1877 1928 1979 2030 2081 2132 2183 2234 2285 2336 2387 2438 2489 2540 2591 2642 2693 2744 2795 2846 2897 2948 2999) (xs1 49 99 149 199 249 299 349 399 449 500 551 602 653 704 755 806 857 908 959 1010 1061 1112 1163 1214 1265 1316 1367 1418 1469 1520 1571 1622 1673 1724 1775 1826 1877 1928 1979 2030 2081 2132 2183 2234 2285 2336 2387 2438 2489 2540 2591 2642 2693 2744 2795 2846 2897 2948 2999) (xs2 49 99 149 199 249 299 349 399 449 500 551 602 653 704 755 806 857 908 959 1010 1061 1112 1163 1214 1265 1316 1367 1418 1469 1520 1571 1622 1673 1724 1775 1826 1877 1928 1979 2030 2081 2132 2183 2234 2285 2336 2387 2438 2489 2540 2591 2642 2693 2744 2795 2846 2897 2948 2999) (increase 0 0 0 0 0 0 0 0 0 0 0 

In [13]:
(show-query-means results 0 #:ylabel "slope")

(c3-data . #hasheq((data . #hasheq((type . line) (xs . #hasheq((slope . xs1))) (columns . ((xs1 500 551 602 653 704 755 806 857 908 959 1010 1061 1112 1163 1214 1265 1316 1367 1418 1469 1520 1571 1622 1673 1724 1775 1826 1877 1928 1979 2030 2081 2132 2183 2234 2285 2336 2387 2438 2489 2540 2591 2642 2693 2744 2795 2846 2897 2948 2999) (slope 3.2487800032437875 3.116550323255408 3.0417076065197968 2.885096186737485 2.8886059603007275 2.86159325877798 2.842298471976018 2.8278273818745463 2.816572089573401 2.8257005830208644 2.8627290158132492 2.876758603929575 2.880386849073603 2.887584867037767 2.9083785214804028 2.9155712688726445 2.9177690294213248 2.943059047203106 2.9456016940650014 2.9475716788878072 2.9579360247498965 2.981753129151218 2.9946674554828245 2.9766477243603853 2.9605135563016005 2.9620774533246883 2.9692449593955637 2.9605858632845323 2.964997863077103 2.98332971599069 2.9735313374277688 2.969208127903361 2.966688418788178 2.9583469010240653 2.9589089357672713 2.95908

In [14]:
(show-query-means results 1 #:ylabel "intercept")

(c3-data . #hasheq((data . #hasheq((type . line) (xs . #hasheq((intercept . xs1))) (columns . ((xs1 500 551 602 653 704 755 806 857 908 959 1010 1061 1112 1163 1214 1265 1316 1367 1418 1469 1520 1571 1622 1673 1724 1775 1826 1877 1928 1979 2030 2081 2132 2183 2234 2285 2336 2387 2438 2489 2540 2591 2642 2693 2744 2795 2846 2897 2948 2999) (intercept 10.155609843312995 12.725763946312489 14.078535412335087 14.743593063627555 15.354116724115887 15.115039093686642 15.079968063035604 15.710457972213632 15.783515784569191 15.310326669254001 14.834689286645556 14.436776957298237 14.402843866666704 14.346330766660122 14.052394740293693 13.778444573661028 13.696796414737365 13.239522093213287 13.19411503823183 12.977527400504997 13.051762458502234 12.554526002203234 12.656685511240134 12.726270920358395 13.053287985674954 13.004405143161552 13.047325073749901 13.142448970957856 13.292577684232263 13.15444376090183 13.27256017049568 13.4230642039189 13.346455342837832 13.491289079952312 13.4541

In [15]:
(show-query-means results 2 #:ylabel "noise-sigma")

(c3-data . #hasheq((data . #hasheq((type . line) (xs . #hasheq((noise-sigma . xs1))) (columns . ((xs1 500 551 602 653 704 755 806 857 908 959 1010 1061 1112 1163 1214 1265 1316 1367 1418 1469 1520 1571 1622 1673 1724 1775 1826 1877 1928 1979 2030 2081 2132 2183 2234 2285 2336 2387 2438 2489 2540 2591 2642 2693 2744 2795 2846 2897 2948 2999) (noise-sigma 20.600827976995195 21.18479792369923 20.009355385878504 18.88659509483861 19.349919600897625 19.30292872334037 18.58941575595272 18.660612453598713 18.352956884251164 18.1154294837941 18.089199295241393 17.883857118495072 17.578503700767786 17.830476577696462 17.756211009371654 17.901206510433653 17.86538348732344 17.854494544749592 17.85065252133325 17.904202902420653 17.95265324721402 17.863358318168057 17.86395910736107 17.983281489573233 17.879442688601642 17.981643715749883 17.913774587302846 17.86209085796434 17.81397152375263 17.79589053476881 17.764743255168597 17.694395159159992 17.651848526054717 17.606991294566246 17.78939827

##Visualizing posterior samples

With a little bit of racket hacking, we can superimpose regression lines defined by our sampled slope and intercept values on the observed data points.

In [16]:
(let ([x-min 0]
      [x-max 29]
      [s (just-the-samples results)])
    (let ([rxs (for/list ([i 50])
                (list x-min x-max))]
          [rys (map 
                (lambda (smp) (list 
                               (+ (vector-ref smp 1) (* x-min (vector-ref smp 0)))
                               (+ (vector-ref smp 1) (* x-max (vector-ref smp 0)))))
               (just-the-samples results))])
      (multi-reg-line-plus-scatter-c3 rxs rys (vector->list xs) (vector->list ys))))

(c3-data . #hasheq((data . #hasheq((type . line) (types . #hasheq((true-ys . scatter))) (xs . #hasheq((true-ys . true-xs) (ys16 . xs16) (ys1 . xs1) (ys0 . xs0) (ys2 . xs2) (ys3 . xs3) (ys4 . xs4) (ys5 . xs5) (ys6 . xs6) (ys7 . xs7) (ys8 . xs8) (ys9 . xs9) (ys10 . xs10) (ys11 . xs11) (ys12 . xs12) (ys13 . xs13) (ys14 . xs14) (ys15 . xs15) (ys17 . xs17) (ys18 . xs18) (ys19 . xs19) (ys20 . xs20) (ys21 . xs21) (ys22 . xs22) (ys23 . xs23) (ys24 . xs24) (ys25 . xs25) (ys26 . xs26) (ys27 . xs27) (ys28 . xs28) (ys29 . xs29) (ys30 . xs30) (ys31 . xs31) (ys32 . xs32) (ys33 . xs33) (ys34 . xs34) (ys35 . xs35) (ys36 . xs36) (ys37 . xs37) (ys38 . xs38) (ys39 . xs39) (ys40 . xs40) (ys41 . xs41) (ys42 . xs42) (ys43 . xs43) (ys44 . xs44) (ys45 . xs45) (ys46 . xs46) (ys47 . xs47) (ys48 . xs48) (ys49 . xs49))) (columns . ((true-xs 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29) (true-ys 36.69636581604345 15.275191354648758 24.603086668962312 29.38871759975995 -6.44337642

In [12]:
(dist-pdf (beta-dist 2 1) 0.3)

0.5999999999999999

In [19]:
(let ([pts (range 0.0001 0.9999 0.001)])
  (line-c3 pts (map (lambda (x) (dist-pdf (beta-dist 2 1) x)) pts)))

(c3-data . #hasheq((data . #hasheq((type . line) (xs . #hasheq((ys1 . xs1))) (columns . ((xs1 0.0001 0.0011 0.0021000000000000003 0.0031000000000000003 0.0041 0.0051 0.0061 0.0071 0.0081 0.0091 0.010100000000000001 0.011100000000000002 0.012100000000000003 0.013100000000000004 0.014100000000000005 0.015100000000000006 0.016100000000000007 0.017100000000000008 0.01810000000000001 0.01910000000000001 0.02010000000000001 0.02110000000000001 0.022100000000000012 0.023100000000000013 0.024100000000000014 0.025100000000000015 0.026100000000000016 0.027100000000000016 0.028100000000000017 0.029100000000000018 0.03010000000000002 0.03110000000000002 0.03210000000000002 0.03310000000000002 0.03410000000000002 0.03510000000000002 0.03610000000000002 0.03710000000000002 0.03810000000000002 0.039100000000000024 0.040100000000000025 0.041100000000000025 0.042100000000000026 0.04310000000000003 0.04410000000000003 0.04510000000000003 0.04610000000000003 0.04710000000000003 0.04810000000000003 0.0491